In [ ]:
!pip install fastapi uvicorn pandas faiss-cpu sentence-transformers transformers torch


In [ ]:
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import pandas as pd
import faiss
import pickle
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import nest_asyncio
import uvicorn


c:\Users\DELL\Desktop\ai_recom\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# This allows FastAPI to run inside Jupyter/Colab notebooks
nest_asyncio.apply()


In [ ]:
app = FastAPI(title="AI Product Recommendation API")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


In [ ]:
import os
print(os.getcwd())


c:\Users\DELL\Desktop\ai_recom\backend


In [ ]:
import os
os.chdir("..")  # move up one level from notebooks/ to ai_recom/
print("Current working directory:", os.getcwd())


Current working directory: c:\Users\DELL\Desktop\ai_recom


In [ ]:
df = pd.read_csv("data/cleaned_products.csv")
index = faiss.read_index("models/faiss_index.bin")
with open("models/meta.pkl", "rb") as f:
    meta = pickle.load(f)


In [ ]:
# Load product dataset
df = pd.read_csv("data/cleaned_products.csv")

# Load FAISS index
index = faiss.read_index("models/faiss_index.bin")

# Load metadata
with open("models/meta.pkl", "rb") as f:
    meta = pickle.load(f)

# Embedding model
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

# Text generation model
tok = AutoTokenizer.from_pretrained("google/flan-t5-base")
gen_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base").to(
    "cuda" if torch.cuda.is_available() else "cpu"
)


In [ ]:
class Query(BaseModel):
    query: str
    top_k: int = 5

@app.post("/recommend")
def recommend(req: Query):
    q_emb = embed_model.encode([req.query], convert_to_numpy=True)
    faiss.normalize_L2(q_emb)
    D, I = index.search(q_emb, req.top_k)
    
    results = []
    for idx, score in zip(I[0], D[0]):
        row = df.iloc[idx].to_dict()
        row['score'] = float(score)
        results.append(row)
    return {"query": req.query, "results": results}


In [ ]:
class GenReq(BaseModel):
    uniq_id: str

@app.post("/generate-description")
def gen_desc(req: GenReq):
    item = df[df['uniq_id'].astype(str) == req.uniq_id]
    if item.empty:
        raise HTTPException(status_code=404, detail="Product not found")
    row = item.iloc[0]
    
    prompt = f"Write a creative marketing description for a {row['categories']}. Title: {row['title']}. Details: {row['description']}"
    inputs = tok(prompt, return_tensors="pt", truncation=True).to(gen_model.device)
    outputs = gen_model.generate(**inputs, max_length=120, num_beams=4)
    gen_text = tok.decode(outputs[0], skip_special_tokens=True)
    
    return {"uniq_id": req.uniq_id, "generated": gen_text}


In [ ]:
@app.get("/analytics")
def analytics():
    stats = {
        "count": len(df),
        "price_mean": float(df['price'].mean()),
        "top_categories": df['categories'].value_counts().head(10).to_dict(),
        "top_brands": df['brand'].value_counts().head(10).to_dict(),
    }
    return stats


In [ ]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()


config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
server = uvicorn.Server(config)

await server.serve()


INFO:     Started server process [27020]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:51929 - "OPTIONS /recommend HTTP/1.1" 200 OK
INFO:     127.0.0.1:51929 - "POST /recommend HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\DELL\Desktop\ai_recom\.venv\Lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\DELL\Desktop\ai_recom\.venv\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\DELL\Desktop\ai_recom\.venv\Lib\site-packages\fastapi\applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\DELL\Desktop\ai_recom\.venv\Lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\DELL\Desktop\ai_recom\.venv\Lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  Fil

INFO:     127.0.0.1:59399 - "OPTIONS /recommend HTTP/1.1" 200 OK
INFO:     127.0.0.1:55478 - "OPTIONS /recommend HTTP/1.1" 200 OK
INFO:     127.0.0.1:55478 - "POST /recommend HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\DELL\Desktop\ai_recom\.venv\Lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\DELL\Desktop\ai_recom\.venv\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\DELL\Desktop\ai_recom\.venv\Lib\site-packages\fastapi\applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\DELL\Desktop\ai_recom\.venv\Lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\DELL\Desktop\ai_recom\.venv\Lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  Fil

INFO:     127.0.0.1:59399 - "POST /recommend HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\DELL\Desktop\ai_recom\.venv\Lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\DELL\Desktop\ai_recom\.venv\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\DELL\Desktop\ai_recom\.venv\Lib\site-packages\fastapi\applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\DELL\Desktop\ai_recom\.venv\Lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\DELL\Desktop\ai_recom\.venv\Lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  Fil

INFO:     127.0.0.1:52226 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:57434 - "GET /analytics HTTP/1.1" 200 OK
INFO:     127.0.0.1:58626 - "GET /analytics HTTP/1.1" 200 OK
INFO:     127.0.0.1:58626 - "GET /analytics HTTP/1.1" 200 OK
INFO:     127.0.0.1:58626 - "GET /analytics HTTP/1.1" 200 OK
INFO:     127.0.0.1:63414 - "GET /analytics HTTP/1.1" 200 OK
INFO:     127.0.0.1:63414 - "GET /analytics HTTP/1.1" 200 OK
